In [ ]:
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf, params

import matplotlib.pyplot as plt
import numpy as np

import os

from src.caiman_preprocessing import copy_data, find_local_max, replace_rows
from src.hyperparams import Hyperparams

## Hyperparameter Setup

In [ ]:
# Hyperparameters for F147
F147 = Hyperparams(name='F147')
F147.set_paths(
    path_orig='data/2p_raw/F147/F147_20210526_fish4_blk1_LT_9dpf_00001.tif',
    path_src='results/F147.tif'
)
F147.set_params_dict(tau=3)
F147.set_lr_params(local_max_thr=20, local_max_rad=50,
                   channel_thr=0, correction_thr=25, correction_rad=50)
F147.set_lr_proxy_params(proxy_slices=[(slice(247, 311), slice(39, 152))])
F147.set_piecewise_processing(proc_slices=[
    (slice(0, 247), slice(0, 256)),
    (slice(247, 320), slice(0, 256))
])

In [ ]:
# Hyperparameters for F201
F201 = Hyperparams(name='F201')
F201.set_paths(
    path_orig='data/2p_raw/F201/F201_20210812_fish2_blk1_RT_9dpf_00001.tif',
    path_src='results/F201.tif'
)
F201.set_params_dict(tau=8)
F201.set_lr_params(local_max_thr=50, local_max_rad=50,
                   channel_thr=0, correction_thr=55, correction_rad=50)

In [ ]:
# Currently selected hyperparameters
hyp = F147

# Select the piece to run CNMF on (only for piecewise processing)
if hyp.piecewise_proc:
    hyp.proc_index = 0

## File Setup

In [ ]:
# Move from the scripts directory to the main project directory
os.chdir('../')

# Set the path to the main results folder
results_path = 'results/'

In [ ]:
# Uncomment the last line to load and resave the data
# See GitHub Issue #377 - https://github.com/flatironinstitute/CaImAn/issues/377#issuecomment-426740429
# copy_data(hyp.path_orig, hyp.path_src)

## CaImAn Parameter Setup

In [ ]:
# Save the CaImAn parameter dictionary
hyp.set_fname(hyp.path_src)
opts = params.CNMFParams(params_dict=hyp.params_dict)

## Movie

In [ ]:
# Play the movie of the original data
movie_orig = cm.load(hyp.path_src)
movie_orig.play()

## Fluorescence Diagnostics

In [ ]:
# Create an array used for diagonistics and line removal
movie_dgn = np.copy(movie_orig)

# Remove all rectangular slices if line removal by proxy is used
if hyp.lr_proxy:
    for rectangle in hyp.proxy_slices:
        movie_dgn[:, rectangle[0], rectangle[1]] = np.full_like(movie_dgn[:, rectangle[0], rectangle[1]], np.nan)

In [ ]:
# Find the mean flourescence of each frame
movie_dgn_means = np.nanmean(movie_dgn, axis=(1, 2))

In [ ]:
# Create a diagonstic plot of the mean fluorescences
plt.plot(movie_dgn_means, '.')
local_max_threshold = plt.axhline(hyp.local_max_thr, c='y')
plt.title("Mean Fluorescences of All Frames")
plt.xlabel("Frame")
plt.ylabel("Mean Fluorescence")
plt.legend([local_max_threshold], ["Local Maximum\nThreshold"], loc=1)
plt.show()

In [ ]:
# Find all local maxima of the mean flourescences
local_max = find_local_max(movie_dgn_means, hyp.local_max_thr, hyp.local_max_rad)
print(local_max)

In [ ]:
# Create diagonstic plots of mean fluorescences around each local maximum
for point in local_max:
    lb, ub = point - hyp.correction_rad, point + hyp.correction_rad + 1
    plt.plot(range(lb, ub), movie_dgn_means[lb:ub], '.')
    channel_threshold = plt.axhline(hyp.channel_thr, c='g')
    plt.title("Mean Fluorescences of Frames Near " + str(point))
    plt.xlabel("Frame")
    plt.ylabel("Mean Fluorescence")
    plt.legend([channel_threshold], ["Channel\nThreshold"], loc=1)
    plt.show()

In [ ]:
# Create diagonstic plots of mean row fluorescences for each image requiring correction
for point in local_max:
    lb, ub = point - hyp.correction_rad, point + hyp.correction_rad + 1
    for i in range(lb, ub):
        if not movie_dgn_means[i] > hyp.channel_thr:
            continue
        frame_row_means = np.nanmean(movie_dgn[i], axis=1)
        for j in range(frame_row_means.size):
            if frame_row_means[j] > hyp.correction_thr:
                plt.plot(frame_row_means, '.')
                correction_threshold = plt.axhline(hyp.correction_thr, c='r')
                plt.title("Mean Row Fluorescences of Frame " + str(i))
                plt.xlabel("Row Index")
                plt.ylabel("Mean Fluorescence")
                plt.legend([correction_threshold], ["Correction\nThreshold"], loc=1)
                plt.show()
                break

## Line Removal

In [ ]:
# Reload the movie and remove lines
movie_lr = cm.load(hyp.path_src)
for point in local_max:
    replace_rows(movie_lr, movie_dgn, point, hyp.channel_thr,
                 hyp.correction_thr, hyp.correction_rad)

In [ ]:
# Create a diagonstic plot of the corrected mean fluorescences
plt.plot(np.mean(movie_lr, axis=(1, 2)), '.')
plt.title("Corrected Mean Fluorescences of All Frames")
plt.xlabel("Frame")
plt.ylabel("Mean Fluorescence")
plt.show()

In [ ]:
# Save the corrected movie
path_lr = results_path + hyp.name + '_lr.tif'
movie_lr.save(path_lr)

## Motion Correction

In [ ]:
# Start or restart the cluster
if 'dview' in locals():
    cm.stop_server(dview=dview)
dview, n_processes = cm.cluster.setup_cluster()[1:3]

In [ ]:
# Perform rigid motion correction
mc = MotionCorrect(path_lr, dview=dview, **opts.get_group('motion'))
mc.motion_correct(save_movie=True)

In [ ]:
# Compare the motion-corrected data with the line-removed data
movie_mc = cm.load(mc.mmap_file)
cm.concatenate([movie_lr - mc.min_mov * mc.nonneg_movie, movie_mc], axis=2).play()

##  Piecewise Processing

In [ ]:
# Get the entire FOV or a subrectangle of the entire FOV
movie_mc_piece = movie_mc
if hyp.piecewise_proc:
    hslice = hyp.proc_slices[hyp.proc_index][0]
    vslice = hyp.proc_slices[hyp.proc_index][1]
    movie_mc_piece = movie_mc[:, hslice, vslice]

In [ ]:
# Switch to the results directory (debug)
os.chdir(results_path)

In [ ]:
# Save the selection
mmap_file = movie_mc_piece.save(hyp.name + '_' + str(hyp.proc_index) + '.mmap')

## Memory Mapping

In [ ]:
# Save memory mapped files from piecewise processing
base_name = hyp.name + '_' + str(hyp.proc_index) + '_memmap_'
fname_mmap = cm.save_memmap([mmap_file], base_name=base_name, order='C', dview=dview)

In [ ]:
# Load memory mapped files from piecewise processing
Yr, dims, T = cm.load_memmap(fname_mmap)
images = np.reshape(Yr.T, [T] + list(dims), order='F')

## Source Extraction

In [ ]:
# Restart the cluster to clean up memory
if 'dview' in locals():
    cm.stop_server(dview=dview)
dview, n_processes = cm.cluster.setup_cluster()[1:3]

In [ ]:
# Update the parameters to use the memory mapped file
hyp.set_fname(fname_mmap)
opts = params.CNMFParams(params_dict=hyp.params_dict)

In [ ]:
# Run CNMF
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit(images)

In [ ]:
# Compute the correlation image
corr_img = cm.local_correlations(images.transpose((1, 2, 0)))
corr_img[np.isnan(corr_img)] = 0

In [ ]:
# Plot the contours of any identified components
cnm.estimates.plot_contours_nb(img=corr_img)

## Component Evaluation

In [ ]:
# Evaluate the quality of inferred spatial components
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

In [ ]:
# Display plot of all components
cnm.estimates.plot_contours_nb(img=corr_img, idx=cnm.estimates.idx_components)

In [ ]:
# Display plot of accepted components
cnm.estimates.nb_view_components(img=corr_img, idx=cnm.estimates.idx_components)

In [ ]:
# Display plot of rejected components, if any
if len(cnm.estimates.idx_components_bad) > 0:
    cnm.estimates.nb_view_components(img=corr_img, idx=cnm.estimates.idx_components_bad)

In [ ]:
# Keep only the accepted components
cnm.estimates.select_components(use_object=True)

## Final Results

In [ ]:
# View a plot of the results
cnm.estimates.nb_view_components(img=corr_img)

In [ ]:
# View a movie of the results
cnm.estimates.play_movie(images)

In [ ]:
# Save results
cnm.save(cnm.mmap_file[:-4] + 'hdf5')

In [ ]:
# Stop the cluster
cm.stop_server(dview=dview)